In [ ]:
from torch.nn.utils.rnn import pad_sequence #(1)

def custom_collate(data): #(2)
    inputs = [torch.tensor(d['tokenized_input']) for d in data] #(3)
    labels = [d['label'] for d in data]

    inputs = pad_sequence(inputs, batch_first=True) #(4)
    labels = torch.tensor(labels) #(5)

    return { #(6)
        'tokenized_input': inputs,
        'label': labels
    }

loader = DataLoader(nlp_data, batch_size=2, shuffle=False, collate_fn=custom_collate) #(7)

iter_loader = iter(loader)
batch1 = next(iter_loader)
pprint(batch1)
batch2 = next(iter_loader)
pprint(batch2)

# {'label': tensor([0, 0]),
#  'tokenized_input': tensor([
#   [  1,   4,   5,   9,   3,   2,   0,   0,   0],
#   [  1,   7,   3,  14,  48,   7,  23, 154,   2]
# ])}

# {'label': tensor([1, 0]),
#  'tokenized_input': tensor([
#   [  1,  30,  67, 117,  21,  15,   2],
#   [  1,  17,   2,   0,   0,   0,   0]])}

Pad the whole dataset to the longest example.

Pad dynamically during batch creation.

In [ ]:
# transform data with collate_fn()

# def collate_fn(data: List[Tuple[torch.Tensor, torch.Tensor]]):
#     tensors, targets = zip(*data)
#     features = pad_sequence(tensors, batch_first=True)
#     targets = torch.stack(targets)
#     return features, targets

In [ ]:
def custom_collate_fn(data: List[Dict[str, str]], tokenizer: PreTrainedTokenizer) -> Dict[str, torch.Tensor]:
    # Ensure tokenizer has a padding token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Extract and concatenate informal and formal statements
    sequences = []
    for idx, example in enumerate(data):
        # Handle null values
        informal = example.get("generated informal statement", "") or ""
        formal = example.get("formal statement", "") or ""

        # Skip if both are empty
        if not informal and not formal:
            continue

        sequences.append(f'informal statement {informal} formal statement {formal}')

    # Tokenize the sequences
    tokenized_data = tokenizer(sequences, padding='longest', truncation=True, return_tensors='pt')
    tokenized_data["labels"] = tokenized_data["input_ids"].clone()

    return tokenized_data

In [ ]:
sample_data = [train_dataset[i] for i in range(batch_size)]
processed_data = custom_collate_fn(sample_data, tokenizer=tokenizer)